In [2]:
import sys
import numpy as np 
import pandas as pd
import warnings
import sympy 
import matplotlib as plt
warnings.filterwarnings('ignore')

from copy import deepcopy
from ipywidgets import IntProgress
from IPython.display import display

sys.path.append('./')
sys.path.append('./Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par

#File related libraries
from expression_writter import *

#Writing files libraries
import re
from file_creator import *
import os
import pickle

In [5]:
ProbTreeFile = "./KL/Probabilty_Trees/BMS_With_prior_nv2_np20.txt_GREEDY"
ExpressionFile = "./KL/Generated_expressions/BMS_With_prior_nv2_np20.txt_GREEDY_Expressions"
directoryPrior = "./Prior/"

In [26]:
#Function to count the number of appearences of an operation in an expression

def count_ops(tree):
    operations = OPS.keys()
    counter = {op: 0 for op in operations}

    #Look for operations
    def run(node):
        if node is None:
            return
        if node.value in counter:
            counter[node.value] += 1

        #now run through offspring
        for offs in node.offspring:
            run(offs)
    run(tree.root)
    return counter
            

#Functions in order to look for the appropiate prior file
def extract_nv_np(filename):
    match = re.search(r'nv(\d+).*?np(\d+)', filename)
    if match:
        return match.group(1), match.group(2)
    return None, None

def find_file(directoryPrior, filename):
    nv, np = extract_nv_np(filename)
    if nv is None or np is None:
        return None

    pattern = rf'nv{nv}\.np{np}\.'

    #obtain all the files from the directory
    fileList = os.listdir(directoryPrior)

    for file in fileList:
        if re.search(pattern, file):
            return os.path.join(directoryPrior, file)
    return None

def create_prior_dict(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        ops = lines[0].strip().split()
        values = list(map(float,lines[1].strip().split()))
        prior_dict = dict(zip(ops,values))
    return prior_dict

def prior_dict(directoryPrior, filename):
    priorFile = find_file(directoryPrior, filename)
    return create_prior_dict(priorFile)

def retrieve_coef(priorDict, operation):
    key1 = "Nopi_" + operation
    key2 = "Nopi2_" + operation

    a , b = priorDict[key1], priorDict[key2]
    return a,b

priorDict = prior_dict(directoryPrior= directoryPrior, filename = ExpressionFile)
#print(priorDict)
print(retrieve_coef(priorDict, "sin"))

def bms_prob(expr, priorDict):
    tree_exp = Tree(from_string = expr)
    counter = count_ops(tree_exp)
    minusLog = 0

    for key, value in counter.items():
        if value > 0:
            a,b = retrieve_coef(priorDict, key)
            minusLog += a*value + b*value**2
    return np.exp(-(minusLog))

def bms_energy(expr, priorDict):
    tree_exp = Tree(from_string = expr)
    counter = count_ops(tree_exp)
    minusLog = 0

    for key, value in counter.items():
        if value > 0:
            a,b = retrieve_coef(priorDict, key)
            minusLog += a*value + b*value**2 #Energy/Description length
    return minusLog
    
expression = "sinh(((pow2((_a8_ + _a12_)) * pow3(pow3(_a9_))) + (_a3_ * _a12_)))"
bms_energy(expression, priorDict)

(6.69508906069221, 0.0010958458372284053)


50.3571501460169

In [24]:
from sympy.parsing.latex import parse_latex
from sympy import symbols, sqrt, log


def subcustom_format(expr):
    """
    Convert a Sympy expression to BMS-friendly expression
    """
    if expr.is_Atom:  # If it is a number or a variable, return without changes
        return str(expr)
    
    if expr.is_Function:  # For functions s.a. sqrt, sin, log, which have argument ()
        func_name = expr.func.__name__
        args = [custom_format(arg) for arg in expr.args]
        
        # Handle of logarithms, so that the base is not specified.
        if func_name == "log" and len(args) == 2:
            return f"log({args[0]})"
        
        return f"{func_name}({', '.join(args)})"
    
    if expr.is_Add or expr.is_Mul:  # Sumas y productos
        op = ' + ' if expr.is_Add else ' * '
        formatted = op.join(subcustom_format(arg) for arg in expr.args)
        return f"({formatted})"
    
    if expr.is_Pow:  # Potencias (A**B)
        base, exp = expr.args
        return f"({subcustom_format(base)} ** {subcustom_format(exp)})"
    
    if expr.is_Div:  # División (A/B)
        num, den = expr.args
        return f"({subcustom_format(num)} / {subcustom_format(den)})"
    
    return str(expr)

def custom_format(latex_expr):
    sympy_expr = parse_latex(latex_expr)
    formatted_expr = subcustom_format(sympy_expr)
    return formatted_expr

In [25]:
a = custom_format(r"\sqrt{\frac{K+(4/3)\mu}{\rho}}")
newTree = Tree(from_string=a)

In [22]:
#recorrer arbre
def recorrer(arrel):
    llista = [ele.value for ele in arrel.offspring]
    print(f"{arrel.value}" + f"{llista}")
    [recorrer(ele) for ele in arrel.offspring]    

    
recorrer(newTree.root)

**['*', '1/2']
*['+', '**']
+['K', '*']
K[]
*['*', 'mu']
*['4', '**']
4[]
**['3', '-1']
3[]
-1[]
mu[]
**['rho', '-1']
rho[]
-1[]
1/2[]


In [27]:
bms_energy(a, priorDict)

45.56349242741382